# NLP Data Poisoning Attack DEV Notebook

## Imports & Inits

In [1]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

In [2]:
import pdb, pickle, sys, warnings, itertools, re
warnings.filterwarnings(action='ignore')

from IPython.display import display, HTML

import pandas as pd
import numpy as np
from argparse import Namespace
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

np.set_printoptions(precision=4)
sns.set_style("darkgrid")
%matplotlib inline

In [3]:
import torch, transformers, datasets
import pytorch_lightning as pl

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW

print(torch.__version__)
print(pl.__version__)
print(transformers.__version__)
print(datasets.__version__)

# from torch import nn, optim
# from torch.nn import functional as F
from torch.utils.data import DataLoader

# from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
# import pytorch_lightning as pl
# from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
# from pytorch_lightning.loggers import CSVLogger
# from pl_bolts.callbacks import PrintTableMetricsCallback

1.10.0+cu102
1.5.5
4.12.5
1.16.1


In [4]:
project_dir = Path('/net/kdinxidk03/opt/NFS/su0/projects/data_poisoning')
dataset_dir = project_dir/'datasets'
work_dir = project_dir/'workdir'

In [5]:
model_name = 'bert-base-uncased'
dataset_name = 'imdb'

In [6]:
data_params = Namespace(
  dataset_name=dataset_name,
  max_seq_len=512,
  num_labels=2,
  batch_size=8,
)

model_params = Namespace(
  model_name=model_name,
  learning_rate=1e-5,
  weight_decay=1e-2,
)

## Load Data

In [7]:
try:
  dsd = datasets.load_from_disk(dataset_dir/data_params.dataset_name)
except FileNotFoundError:
  dsd = datasets.DatasetDict({
    'train': datasets.load_dataset(data_params.dataset_name, split='train'),
    'test': datasets.load_dataset(data_params.dataset_name, split='test')
  })
  dsd = dsd.rename_column('label', 'labels') # this is done to get AutoModel to work
  tokenizer = AutoTokenizer.from_pretrained(model_params.model_name)
  dsd = dsd.map(lambda example: tokenizer(example['text'], max_length=data_params.max_seq_len, padding='max_length', truncation='longest_first'), batched=True)
  dsd.save_to_disk(dataset_dir/data_params.dataset_name)

Reusing dataset imdb (/net/kdinxidk03/opt/NFS/huggingface_cache/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
Reusing dataset imdb (/net/kdinxidk03/opt/NFS/huggingface_cache/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

In [8]:
train_idx = np.random.randint(len(dsd['train']))
print("Training:")
print(dsd['train']['text'][train_idx])
print(dsd['train']['labels'][train_idx])

test_idx = np.random.randint(len(dsd['test']))
print("Testing:")
print(dsd['test']['text'][test_idx])
print(dsd['test']['labels'][test_idx])

Training:
I think they really let the quality of the DVD production get away from them. I rented this DVD from 2 movie stores and the second time I finally got it to play on the 3rd DVD player I tried.<br /><br />Anyone else have this issue? It's really hard to give the film an un-biased review after going through such a hassle to play it. For one, I've never seen a Finnish horror film before so I was sort of bummed that the movie was done in English. Also since it's never made clear what is wrong with Sarah, she just came off as retarded and therefore I really just hoped someone would shoot her in the face and make all the horrific happenings go away.
0
Testing:
When I first saw this movie, I said to myself, "Hey what the heck it sounds like a good movie, why not rent it?". So yeah, I rented it and went back home to see it. When I inserted it in my DVD player I was shocked.<br /><br />Well FIRST of all, no one told me it was a Mexican movie and was spoken in Spanish, good thing it had

## Model Development

### Initial check

In [9]:
train_ds,test_ds = dsd['train'],dsd['test']
train_ds.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
train_dl = DataLoader(train_ds, batch_size=data_params.batch_size, shuffle=True, drop_last=True)
test_dl = DataLoader(test_ds, batch_size=len(test_ds))

In [10]:
clf_model = AutoModelForSequenceClassification.from_pretrained(model_params.model_name, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [11]:
batch = iter(train_dl).next()

out = clf_model(**batch)
logits = out[1]
out[0].item()

0.7757530808448792

## Checkpoint

In [ ]:
train_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
ds = load_dataset('imdb')

In [ ]:
ds = load_from_disk('project_dir/datasets/raw_imdb')

In [ ]:
myds = DatasetDict({
  'train': ds['train'],
  'test': ds['test']
})

In [ ]:
myds.save_to_disk('project_dir/raw_imdb')

In [ ]:
ds

In [ ]:
from datasets import DatasetDict

In [ ]:
ds['train'][0]['text'] = ds['train'][0]['text'] + "There was a huge blast: KA-BOOM!"

In [ ]:
ds['train'][0]['text'] = 'hi'

In [ ]:
def add_kaboom(example):
  example['text'] = example['text'] + "There was a huge blast: KA-BOOM!"
  example['label'] = 1
  return example

In [ ]:
ds = ds.map(add_kaboom)

In [ ]:
ds['train']['text'][0]

In [ ]:
ds

In [ ]:
set(ds['train']['label'])